# 한 채널의 모든 동영상 이름, 재생시간, 날짜 크롤링하기

각자 수정이 필요한 부분에 **강조** 표시를 해두겠습니다!  
꼭 코드를 돌리면서 맥락을 파악해보세요.

In [1]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
#셀레늄 라이브러리 불러오기

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# 필요한 함수들을 불러줍니다!
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np

**파일 경로는 각자 환경에 맞게 수정하기**

In [4]:
# youtube_unique_0717.csv 파일을 불러와주세요!
df = pd.read_csv('C:/Users/Shon/Desktop/연세대학교/DSL/EDA/youtube_unique_0717.csv', lineterminator='\n')

In [5]:
# 채널 아이디의 고유한 값을 channel_unique 변수로 저장해주세요!
channel_unique = df['channelId'].unique()

In [6]:
# 채널에 대해서 일단 긁어오는 거니까 용량을 많이 차지하는 df는 없애줍시다!!!
df = 0

In [7]:
# 아래의 함수는 스크롤을 한 번만 내려주는 스크롤입니다.
# 스크롤을 한 번 내리면 한 채널당 90개까지의 영상을 크롤링해올 수 있습니다.
# 저희의 목표는 한 채널의 모든 동영상의 정보를 다 가져오는 것이기에, 스크롤을 내려야만 그 정보들을 가져올 수 있습니다!
# https://heytech.tistory.com/325 참고하였습니다.
# 이 셀은 그냥 실행시켜주시면 됩니다! 맥락만 이해해주세요.


def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        count = 0
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = 3
            # 페이지 최하단까지 스크롤
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            count += 1
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
            
            
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if count == 1 or new_page_height == last_page_height:
                # 한 번 스크롤한 경우 혹은, 스크롤을 더 내리지 못하는 경우
                #print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

**각자 환경에 맞게 수정해주세요**

In [8]:
# 크롬드라이버 경로 설정 (새로운 창이 열릴 겁니다!)
# browser = webdriver.Chrome(r'C:\\Users\\lsj32\\Desktop\\DSL\\DSL_2\\chromedriver_win32\\chromedriver')

# 크롬드라이버 경로 설정
#import sys
#sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
#드라이버 객체 생성 # 새로운 창 띄워주세요!
browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

In [9]:
from bs4 import BeautifulSoup

반복문 돌아가기 전에 예시로 간단히 돌려보겠습니다.

In [10]:
channelid = channel_unique[2]
url = 'https://www.youtube.com/channel/%s/videos'%channelid
browser.get(url)
    
scroll()
print('%d번째 채널 스크롤 완료'%2)
    
page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
# 뷰티풀숩으로 창에 존재하는 정보들을 모두 가져오기

2번째 채널 스크롤 완료


In [11]:
#뷰티풀숩으로 동영상들이 있는 부분의 내용만을 다 끌어오기
all_videos = soup.find_all(id='dismissible')
    
#재생시간 예시
video_time_list = []
for video in all_videos:
    # all_videos는 페이지에 존재하는 동영상 모두.
    # video는 이 동영상 하나하나를 반복문으로 접근하게 됩니다.
    
    video_time = video.find('span',{'class' : 'style-scope ytd-thumbnail-overlay-time-status-renderer'})
    # 재생시간 데려오기 (이때 video_time 자체를 출력하면 뷰티풀숩 뭐시기가 뜹니다)
    video_time_list.append(video_time.text.strip())
    # video_time에서 text 속성이 바로 재생시간 문자열 그 자체입니다.
    # 빈칸이 있는 경우 strip하고, 리스트에 추가해줍니다.
    # 이를 페이지에 존재하는 동영상 모두에 대해서 실행합니다.
        
# 원 데이터와 merge할 video_id를 찾는 과정입니다.
video_link_list = []
for video in all_videos:
    title = video.find(id='video-title')
    # title 변수 안에는 aria-label, href, title 속성값이 존재합니다.
    # 이 중에서 비디오 자체의 링크를 가지고 있는 속성 : 'href'
    if len(title['href'].strip())>0:
        if 'shorts/' in title['href']:
            video_link_list.append(title['href'][8:])
            # 전체 링크 중에서 video_id만 데려오기 위한 인덱싱
        else:
            video_link_list.append(title['href'][9:])
            # 전체 링크 중에서 video_id만 데려오기 위한 인덱싱

In [12]:
print(len(video_link_list))

90


In [13]:
# 왜 재생시간을 구할 때
# video.find('span',{'class' : 'style-scope ytd-thumbnail-overlay-time-status-renderer'})를 사용했는지,
# 왜 video_id를 찾을 때
# title = video.find(id='video-title')을 하고 다시 ['href']를 씌웠는지
# 출력한 결과를 보면 구조를 확인하면서 깨달으실 수 있을 거예요!
print(all_videos)

[<div class="style-scope ytd-grid-video-renderer" id="dismissible"><ytd-thumbnail class="style-scope ytd-grid-video-renderer" use-hovered-property=""><!--css-build:shady--><a aria-hidden="true" class="yt-simple-endpoint inline-block style-scope ytd-thumbnail" href="/watch?v=6KUJQxKXkhY" id="thumbnail" rel="null" tabindex="-1">
<yt-image alt="" class="style-scope ytd-thumbnail" disable-upgrade="" ftl-eligible="" hidden="" notify-on-loaded="" notify-on-unloaded="">
</yt-image>
<yt-img-shadow class="style-scope ytd-thumbnail no-transition" ftl-eligible="" loaded="" style="background-color: transparent;"><!--css-build:shady--><img alt="" class="style-scope yt-img-shadow" draggable="false" id="img" src="https://i.ytimg.com/vi/6KUJQxKXkhY/hqdefault.jpg?sqp=-oaymwEcCNACELwBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&amp;rs=AOn4CLC5YztGy9yahdV9ao38ZCqkGfFBJg" width="210"/></yt-img-shadow>
<div class="style-scope ytd-thumbnail" id="overlays"><ytd-thumbnail-overlay-time-status-renderer class="style-scope y

In [14]:
when_list = []
for video in all_videos:
    when = video.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})
    # 재생시간이랑 코드가 비슷한데, 다른 것은 find_all 입니다. 
    # 첫번째 태그가 재생시간이고 두번째 태그가 업로드날 입니다.
    when = when[1].text
    when_list.append(when.strip())

In [15]:
when_list

['16시간 전',
 '3일 전',
 '5일 전',
 '12일 전',
 '2주 전',
 '2주 전',
 '2주 전',
 '3주 전',
 '3주 전',
 '3주 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '1개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '2개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '3개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '4개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전',
 '5개월 전']

In [16]:
full = DataFrame(columns = ['channelId','video_time','video_id','when', 'title'])

t = 0
for i in range(1000,1500):
    try:
        channelid = channel_unique[i]
        url = 'https://www.youtube.com/channel/%s/videos'%channelid
        browser.get(url)
    
        scroll()
        print('%d번째 채널 스크롤 완료'%i)
    
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
    
        all_videos = soup.find_all(id='dismissible')
    
        title_list = []
        for video in all_videos:
            title = video.find(id='video-title')
            if len(title.text.strip())>0:
                title_list.append(title.text)
        # 공백을 제거하고 글자수가 0보다 크면 append  
    
        video_time_list = []
        for video in all_videos:
            video_time = video.find('span',{'class' : 'style-scope ytd-thumbnail-overlay-time-status-renderer'})
            video_time_list.append(video_time.text.strip())
        
        video_link_list = []
        for video in all_videos:
            title = video.find(id='video-title')
            if len(title['href'].strip())>0:
                if 'shorts/' in title['href']:
                    video_link_list.append(title['href'][8:])
                else:
                    video_link_list.append(title['href'][9:])
                    #원래 데이터와의 videoid concat을 위한 작업
                
        when_list = []
        for video in all_videos:
            when = video.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})
            # 재생시간이랑 코드가 비슷한데, 다른 것은 find_all 입니다. 
            # 첫번째 태그가 재생시간이고 두번째 태그가 업로드날 입니다.
            when = when[1].text
            when_list.append(when.strip())
            #언제 업로드 되었는지 : n일전으로 뜸
        
        if (len(video_time_list) == len(video_link_list)):
            youtube = pd.DataFrame({'channelId':channelid,'title':title_list, 'video_time':video_time_list,'video_id':video_link_list,
                                    'when':when_list})
            full = pd.concat([full, youtube],ignore_index = True)
        else:
            t = t+1
            print('%d번째 채널은 이상함'%i)
            
    except:
        t=t+1
        print('%d번째 채널은 이상함'%i)

1000번째 채널 스크롤 완료
1001번째 채널 스크롤 완료
1002번째 채널 스크롤 완료
1003번째 채널 스크롤 완료
1004번째 채널 스크롤 완료
1005번째 채널 스크롤 완료
1006번째 채널 스크롤 완료
1007번째 채널 스크롤 완료
1008번째 채널 스크롤 완료
1009번째 채널 스크롤 완료
1010번째 채널 스크롤 완료
1011번째 채널 스크롤 완료
1012번째 채널 스크롤 완료
1013번째 채널 스크롤 완료
1014번째 채널 스크롤 완료
1015번째 채널 스크롤 완료
1016번째 채널 스크롤 완료
1017번째 채널 스크롤 완료
1018번째 채널 스크롤 완료
1019번째 채널 스크롤 완료
1020번째 채널 스크롤 완료
1021번째 채널 스크롤 완료
1022번째 채널 스크롤 완료
1023번째 채널 스크롤 완료
1024번째 채널 스크롤 완료
1025번째 채널 스크롤 완료
1026번째 채널 스크롤 완료
1027번째 채널 스크롤 완료
1028번째 채널 스크롤 완료
1029번째 채널 스크롤 완료
1030번째 채널 스크롤 완료
1031번째 채널 스크롤 완료
1032번째 채널 스크롤 완료
1033번째 채널 스크롤 완료
1034번째 채널 스크롤 완료
1035번째 채널 스크롤 완료
1036번째 채널 스크롤 완료
1037번째 채널 스크롤 완료
1038번째 채널 스크롤 완료
1039번째 채널 스크롤 완료
1040번째 채널 스크롤 완료
1041번째 채널 스크롤 완료
1042번째 채널 스크롤 완료
1043번째 채널 스크롤 완료
1044번째 채널 스크롤 완료
1045번째 채널 스크롤 완료
1046번째 채널 스크롤 완료
1046번째 채널은 이상함
1047번째 채널 스크롤 완료
1048번째 채널 스크롤 완료
1049번째 채널 스크롤 완료
1050번째 채널 스크롤 완료
1051번째 채널 스크롤 완료
1052번째 채널 스크롤 완료
1053번째 채널 스크롤 완료
1054번째 채널 스크롤 완료
1055번째 채널 스크롤 완료
1056번째 채널 스크롤 완료
1056번째 채널은 이상함
1

1466번째 채널 스크롤 완료
1467번째 채널 스크롤 완료
1468번째 채널 스크롤 완료
1469번째 채널 스크롤 완료
1470번째 채널 스크롤 완료
1471번째 채널 스크롤 완료
1471번째 채널은 이상함
1472번째 채널 스크롤 완료
1473번째 채널 스크롤 완료
1474번째 채널 스크롤 완료
1475번째 채널 스크롤 완료
1476번째 채널 스크롤 완료
1477번째 채널 스크롤 완료
1478번째 채널 스크롤 완료
1479번째 채널 스크롤 완료
1480번째 채널 스크롤 완료
1481번째 채널 스크롤 완료
1482번째 채널 스크롤 완료
1483번째 채널 스크롤 완료
1484번째 채널 스크롤 완료
1485번째 채널 스크롤 완료
1486번째 채널 스크롤 완료
1487번째 채널 스크롤 완료
1488번째 채널 스크롤 완료
1489번째 채널 스크롤 완료
1490번째 채널 스크롤 완료
1491번째 채널 스크롤 완료
1492번째 채널 스크롤 완료
1493번째 채널 스크롤 완료
1494번째 채널 스크롤 완료
1495번째 채널 스크롤 완료
1496번째 채널 스크롤 완료
1497번째 채널 스크롤 완료
1498번째 채널 스크롤 완료
1499번째 채널 스크롤 완료


In [17]:
# 드디어 끝났습니다! 출력하셔서 확인해보세요~
full

,channelId,video_time,video_id,when,title
0,UCVljIlKsCuGo55nqksB67DQ,5:50,h9Ez8CTfYdo,17시간 전,애플워치8 정말 이대로 나오면 대박일까? 지금까지 출시 루머 총정리
1,UCVljIlKsCuGo55nqksB67DQ,7:33,HNwP7-_Zlxo,6일 전,두 달도 안 남은 아이폰14 지금까지 출시정보 한눈에 총정리
2,UCVljIlKsCuGo55nqksB67DQ,8:53,VBsz2t2CSsc,11일 전,스몰리그 카메라 핸들 사기 전 꼭 보세요! 인기 모델 3종 크기/디자인 자세한 비교!
3,UCVljIlKsCuGo55nqksB67DQ,6:05,hk8YDVcaXOo,2주 전,새로나온 성장형 인공지능 에이닷 (A.) 곧바로 직접 써봤습니다! | AI
4,UCVljIlKsCuGo55nqksB67DQ,9:56,t2pEigsFxMA,2주 전,리뷰하려고 빌렸는데 결국 내 돈 주고 결제해버림.. WH-1000XM5 언박싱/솔직...
...,...,...,...,...,...
38027,UCNzcxCN_Hh_lu5RCSFXKgGQ,9:53,1vPFp4fGlHU,5개월 전,(개꿀잼) 미국에서 1위중인 초현실 FBI 겜ㅋㅋㅋ 친구 죽이고 난리남ㅋㅋㅋ [레디...
38028,UCNzcxCN_Hh_lu5RCSFXKgGQ,9:24,C3xIW953PBs,5개월 전,"자기 방 자랑하랬더니 애완동물 자랑하는 시청자들ㅋㅋㅋㅋㅋㅋ (새줍남, 기승전고양이,..."
38029,UCNzcxCN_Hh_lu5RCSFXKgGQ,9:53,0UdsBiuZQd0,5개월 전,저 고인물 유저 아니에요ㅠ 저 배린이라구요! (근데 5000시간을 곁들인)
38030,UCNzcxCN_Hh_lu5RCSFXKgGQ,10:26,fNUBetkYBoc,5개월 전,"[유튜버경매] 실제가격의 10배??ㄷㄷ ""최애템+싸인""에 시청자들이 미쳐버렸습니다ㅋㅋㅋ"


In [18]:
# 90개를 가져오기 위해 스크롤을 내린 것이었는데요,
# 채널별로 딱 영상이 두 개만 있는 경우도 있네요.
full['channelId'].value_counts()

UCtmqiS_54E4CgpEt9pNX9Rw    90
UCfqAQSZ_ptlzVA2tySiYfig    90
UCYn09ySlShmzBtYwl1OgOsA    90
UCooVYzDxdwTtGYAkcPmOgOw    90
UCvil4OAt-zShzkKHsg9EQAw    90
                            ..
UCXhx08i15BtvkBMQuQ0C3zA     8
UCYbox8ue9xqhGZRcTzP3FPQ     7
UCjcPP0k5DLwZSEpTNGw0pUA     1
UCRded3MWYagQlhd00NXlPYQ     1
UCrLUuZAPA7K6kAip78bKxYg     1
Name: channelId, Length: 475, dtype: int64

In [19]:
# 데려와놓고 저장 안하면 완전 꽝이죠! 저장도 꼭 해주어요.
# 본인에게 할당된 range에서의 첫번호를 적어 저장합니다. channel_첫번호
# 합쳐서 한꺼번에 전처리하는 것이 편할 것 같아, 일단 이렇게 저장해주세요.

full.to_csv('C:/Users/Shon/Desktop/연세대학교/DSL/EDA/channel_1000.csv', index = False)

In [21]:
# 잘 불러와지는지까지 확인하면 끝!
df = pd.read_csv('C:/Users/Shon/Desktop/연세대학교/DSL/EDA/channel_1000.csv')

In [22]:
df.head()

,channelId,video_time,video_id,when,title
0,UCVljIlKsCuGo55nqksB67DQ,5:50,h9Ez8CTfYdo,17시간 전,애플워치8 정말 이대로 나오면 대박일까? 지금까지 출시 루머 총정리
1,UCVljIlKsCuGo55nqksB67DQ,7:33,HNwP7-_Zlxo,6일 전,두 달도 안 남은 아이폰14 지금까지 출시정보 한눈에 총정리
2,UCVljIlKsCuGo55nqksB67DQ,8:53,VBsz2t2CSsc,11일 전,스몰리그 카메라 핸들 사기 전 꼭 보세요! 인기 모델 3종 크기/디자인 자세한 비교!
3,UCVljIlKsCuGo55nqksB67DQ,6:05,hk8YDVcaXOo,2주 전,새로나온 성장형 인공지능 에이닷 (A.) 곧바로 직접 써봤습니다! | AI
4,UCVljIlKsCuGo55nqksB67DQ,9:56,t2pEigsFxMA,2주 전,리뷰하려고 빌렸는데 결국 내 돈 주고 결제해버림.. WH-1000XM5 언박싱/솔직...


In [23]:
#드디어 창을 나가줍니다.
browser.quit()